In [2]:
%cd ../
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from keras.models import load_model, Model, Sequential
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import kneighbors_graph
from sklearn import metrics
from matplotlib import pyplot as plt
from data import get_fft_data

/scratch/sk7898/deep_radar


Using TensorFlow backend.


### Clustering Metrics

Reference: https://gemfury.com/stream/python:scikit-learn/-/content/metrics/cluster/supervised.py

**homogeneity_score**:
A clustering result satisfies homogeneity if all of its clusters contain only data points which are members of a single class.
- Perfect labelings are homogeneous
```python
homogeneity_score([0, 0, 1, 1], [1, 1, 0, 0]) --> 1.0
```
- Non-perfect labelings that further split classes into more clusters can be perfectly homogeneous
```python
homogeneity_score([0, 0, 1, 1], [0, 1, 2, 3]) --> 1.000000
```
- Clusters that include samples from different classes do not make for an homogeneous labeling
```python
homogeneity_score([0, 0, 1, 1], [0, 1, 0, 1]) --> 0.0...
```
***

**completeness_score**:
A clustering result satisfies completeness if all the data points that are members of a given class are elements of the same cluster.
- Perfect labelings are complete
- Non-perfect labelings that assign all classes members to the same clusters are still complete
```python 
completeness_score([0, 0, 1, 1], [0, 0, 0, 0]) --> 1.0 
completeness_score([0, 1, 2, 3], [0, 0, 1, 1]) --> 0.999...
```
- If classes members are split across different clusters, the assignment cannot be complete
```python
completeness_score([0, 0, 1, 1], [0, 1, 0, 1]) --> 0.0
```
***

**mutual_info_score**:
Mutual Information between two clusterings. The Mutual Information is a measure of the similarity between two labels of the same data. Where ${|U_i|}$ is the number of the samples in cluster $U_i$ and ${|V_j|}$ is the number of the samples in cluster $V_j$, the Mutual Information between clusterings U and V is given as:

<center> 
    $MI(U,V)=\sum_{i=1}^{|U|} \sum_{j=1}^{|V|} \frac{|U_i\cap V_j|}{N}\log\frac{N|U_i \cap V_j|}{|U_i||V_j|}$ 
</center>

This metric is furthermore symmetric: switching ``label_true`` with ``label_pred`` will return the same score value. This can be useful to measure the agreement of two independent label assignments strategies on the same dataset when the real ground truth is not known.
    
***

**contingency_matrix:**
Matrix C such that $C_{i,j}$ is the number of samples in true class i and in predicted class j. 

***

**v_measure_score**:
harmonic mean between homogeneity and completeness:
<center>
$v = (1 + beta) * homogeneity * completeness / (beta * homogeneity + completeness)$
</center>

- Perfect labelings are both homogeneous and complete, hence have score 1.0
```python
v_measure_score([0, 0, 1, 1], [1, 1, 0, 0]) --> 1.0
```

- Labelings that assign all classes members to the same clusters are complete be not homogeneous, hence penalized
```python
v_measure_score([0, 0, 1, 2], [0, 0, 1, 1]) --> 0.8...
v_measure_score([0, 1, 2, 3], [0, 0, 1, 1]) --> 0.66...
```

- Labelings that have pure clusters with members coming from the same classes are homogeneous but un-necessary splits harms completeness and thus penalize V-measure as well
```python
v_measure_score([0, 0, 1, 1], [0, 1, 2, 3]) --> 0.66...
```

- If classes members are completely split across different clusters, the assignment is totally incomplete, hence the V-Measure is null
```python
v_measure_score([0, 0, 0, 0], [0, 1, 2, 3]) --> 0.0...
```

- Clusters that include samples from totally different classes totally destroy the homogeneity of the labeling
```python
v_measure_score([0, 0, 1, 1], [0, 0, 0, 0]) --> 0.0...
```

***

**adjusted_rand_score**:
The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.
The raw RI score is then “adjusted for chance” into the ARI score using the following scheme:
<center>
$ARI = (RI - Expected\_RI) / (max(RI) - Expected\_RI)$
</center>

- Labelings that assign all classes members to the same clusters are complete be not always pure, hence penalized
```python
adjusted_rand_score([0, 0, 1, 2], [0, 0, 1, 1]) --> 0.57...
```
- ARI is symmetric, so labelings that have pure clusters with members coming from the same classes but unnecessary splits are penalized
```python
adjusted_rand_score([0, 0, 1, 1], [0, 0, 1, 2]) --> 0.57...
```
- If classes members are completely split across different clusters, the assignment is totally incomplete, hence the ARI is very low
```python
adjusted_rand_score([0, 0, 0, 0], [0, 1, 2, 3]) --> 0.0
```

In [101]:
def get_n_homogeneous_clusters(y_true, cluster_labels, n_clusters):
    homogeneous_clusters = []
    scores = np.zeros(n_clusters, dtype=np.float)
    
    for i in range(n_clusters):
        indexes = cluster_labels == i
        scores[i] = metrics.cluster.homogeneity_score(y_true[indexes], cluster_labels[indexes])
        if scores[i] > 0.8:
            homogeneous_clusters.append(i)

    return homogeneous_clusters

In [69]:
def print_scores(y, cluster_labels, n_clusters=4):
    if y.ndim != 1:
        y = y.flatten()
        
    h_clusters = get_n_homogeneous_clusters(y,
                                            cluster_labels, 
                                            n_clusters=n_clusters)
    
    print("Homogeneity:", metrics.homogeneity_score(y, cluster_labels))
    print('Homogeneous Clusters: ', len(np.unique(h_clusters)))
    print("Mutual Information: ", metrics.mutual_info_score(y, cluster_labels))
    
    if n_clusters == 4:
        print("Completeness:", metrics.completeness_score(y, cluster_labels))
        print("V-measure:", metrics.v_measure_score(y, cluster_labels))
        print("Adjusted Rand-Index: ", metrics.cluster.adjusted_rand_score(y, cluster_labels))   
        print("Contingency Matrix: \n", metrics.cluster.contingency_matrix(y, cluster_labels))

In [4]:
def get_centroid_cluster(X):
    length = X.shape[0]
    sum_x = np.sum(X[:, 0])
    sum_y = np.sum(X[:, 1])
    return sum_x/length, sum_y/length

In [5]:
def get_centroids(X, y, n_classes=2):
    cents = np.empty((n_classes, 2))
    
    for i in range(n_classes):
        X_subset = X[y == i]
        cents[i, :] = get_centroid_cluster(X_subset)
    
    return cents

In [6]:
def get_inc_seqs(seqs, inc):
    inc_seqs = [seqs[idx].split('_') for idx in inc]
    fraction = [x[0] +'_'+ str(np.round(int(x[-1])/int(x[1]), 3)) for x in inc_seqs]
    return fraction

In [7]:
def get_cluster_mode(y, clust_labels, n_clusters=2):
    lbls_lst = [[] for i in range(n_clusters)]
    modes = np.empty((n_clusters), dtype=np.int8)

    for t, p in zip(y, clust_labels):
        lbls_lst[int(p)].append(t)

    for idx, lst in enumerate(lbls_lst):
        modes[idx] = stats.mode(lst, axis=None)[0][0]
        
    return modes

In [8]:
def get_correct_incorrect_idx(y, cls_pred, n_classes=2, print_res=False):
    c_idx = []
    inc_idx = []
    c_lst = [[] for i in range(n_classes)]
    inc_lst = [[] for i in range(n_classes)]

    for idx, (t, pred) in enumerate(zip(y, cls_pred)):
        if t[0] != pred:
            inc_idx.append(idx)
            inc_lst[int(t)].append(idx)
        else:
            c_idx.append(idx)
            c_lst[int(t)].append(idx)

    if print_res:
        if n_classes == 2:
            print('Correct:{}, 1:{}, 2:{}'.format(len(y) - len(inc_idx),
                                                  len(c_lst[0]),
                                                  len(c_lst[1])))
            
            print('Incorrect:{}, 1:{}, 2:{}'.format(len(inc_idx),
                                                    len(inc_lst[0]),
                                                    len(inc_lst[1])))
        elif n_classes == 3:
            print('Correct:{}, 1:{}, 2:{}, 3:{}'.format(len(y) - len(inc_idx),
                                                        len(c_lst[0]),
                                                        len(c_lst[1]),
                                                        len(c_lst[2])))
            
            print('Incorrect:{}, 1:{}, 2:{}, 3:{}'.format(len(inc_idx),
                                                          len(inc_lst[0]),
                                                          len(inc_lst[1]),
                                                          len(inc_lst[2])))           
        elif n_classes == 4:
            print('Correct:{}, 1:{}, 2:{}, 3:{}, 4:{}'.format(len(y) - len(inc_idx),
                                                              len(c_lst[0]),
                                                              len(c_lst[1]),
                                                              len(c_lst[2]),
                                                              len(c_lst[3])))
            
            print('Incorrect:{}, 1:{}, 2:{}, 3:{} 4:{}'.format(len(inc_idx),
                                                               len(inc_lst[0]),
                                                               len(inc_lst[1]),
                                                               len(inc_lst[2]),
                                                               len(inc_lst[3])))
        else:
            print('Number of Classes not supported till now')
        
    return c_idx, inc_idx, c_lst, inc_lst

In [9]:
def get_subset_data(X, y, old_y, idxs):
    X_c = [X[idx] for idx in idxs]
    y_c = [y[idx] for idx in idxs]
    old_y_c = [old_y[idx] for idx in idxs]
    
    return np.array(X_c), np.array(y_c).flatten(), np.array(old_y_c).flatten()

In [10]:
def plot_data(axs, X, y, 
              palette, 
              row_idx, col_idx, 
              y_annotate=None, 
              cents=None, 
              legend='full'):
    
    p1 = sns.scatterplot(X[:,0], X[:,1],
                         hue=y,
                         legend=legend,
                         palette=palette,
                         ax=axs[row_idx, col_idx])

    if y_annotate is not None:
        for i, c in enumerate(cents):
            p1.text(c[0], c[1], 
                    y_annotate[i], 
                    horizontalalignment='left',
                    size='medium',
                    color='black',
                    weight='semibold')

In [11]:
def get_plot_fig(rows, cols=None, col_names=None):
    sns.set(rc={'figure.figsize':(16, 20)})    
    fig, axs = plt.subplots(nrows=rows, ncols=cols)

    if col_names:
        for ax, col in zip(axs[0], col_names):
            ax.set_title(col)   
    
    return fig, axs

In [12]:
def get_relabel_dict(X, X_test, y_test, n_clusters):
    relabel_dict = {1: [0, 0, 0, 0], 2: [0, 0, 0, 0], 3: [0, 0, 0, 0], 4: [0, 0, 0, 0]}

    kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=10)
    kmeans.fit(X)

    clust_labels = kmeans.predict(X_test)
    clust_true_labels = get_cluster_mode(y_test, clust_labels, n_clusters=n_clusters)

    new_labels = [clust_true_labels[cluster_id] for cluster_id in clust_labels]

    for old_y, new_y in zip(y_test, new_labels):
        relabel_dict[old_y][new_y-1] += 1
    
    for key in relabel_dict.keys():
        print('True Label:{} Relabel:{}'.format(key, relabel_dict[key]))  

**DBSCAN**

Density-Based Spatial Clustering of Applications with Noise (DBSCAN) locates regions of high density that
are separated from one another by regions of low density. If there are at least min_samples many data points 
within a distance of eps to a given data point, that data point is classified as a core sample
core samples that are closer to each other than the distance eps are put into the same cluster by DBSCAN.\
*Density at a point P*: Number of points within a circle of Radius *eps (ϵ)* from point P.\
*Dense Region*: For each point in the cluster, the circle with radius ϵ contains at least minimum number of points *min_samples*.

<div>
<img src="dbscan.png" width="500"/>
</div>

In [13]:
def clustering_dbscan(X_train, y_train,
                      min_samples_lst=None,
                      eps_lst=None,
                      pca_train=True):

    pca = PCA(n_components=2).fit(X_train)
    X = pca.transform(X_train)
    X_train = X if pca_train else X_train

    if eps_lst and min_samples_lst:
        col_names = ['min_samples:'+ str(min_samples)+' eps:' + str(eps) for min_samples in min_samples_lst
                                                                         for eps in eps_list 
                    ]
        
        fig, axs = get_plot_fig(rows=len(min_samples_lst),
                                cols=len(eps_lst),
                                col_names=col_names)
        
        for row_idx, min_samples in enumerate(min_samples_lst):
            for col_idx, eps in enumerate(eps_lst):
                db = DBSCAN(eps=eps, min_samples=min_samples).fit(X_train)
                core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
                core_samples_mask[db.core_sample_indices_] = True
                labels = db.labels_
                
                n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
                n_noise_ = list(labels).count(-1)

                print('Estimated number of clusters: %d' % n_clusters_)
                print('Estimated number of noise points: %d' % n_noise_)

                labels_palette = sns.color_palette("bright", len(np.unique(labels)))
        
                plot_data(axs, X, labels,
                          palette=labels_palette,
                          row_idx=row_idx,
                          col_idx=col_idx,
                          legend='full')
            
                print_scores(y_train, labels)
                
    else:
        db = DBSCAN(eps=0.3, min_samples=10).fit(X_train)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_

        # DBSCAN helps us to identify noise in the data.
        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)

        print('Estimated number of clusters: %d' % n_clusters_)
        print('Estimated number of noise points: %d' % n_noise_)
        
        print_scores(y_train, labels)
        labels_palette = sns.color_palette("bright", len(np.unique(labels)))
        
        sns.scatterplot(X[:, 0], X[:, 1], hue=labels, palette=labels_palette)

In [14]:
def clustering_agglomerative(X_train, y_train, 
                             affinity,
                             linkage,
                             n_clusters,
                             pca_train=True):
    
    # Create a graph capturing local connectivity. Larger number of neighbors
    # will give more homogeneous clusters to the cost of computation
    # time. A very large number of neighbors gives more evenly distributed
    # cluster sizes, but may not impose the local manifold structure of
    # the data
    
    pca = PCA(n_components=2).fit(X_train)
    X = pca.transform(X_train)
    
    X_train = X if pca_train else X_train
    
    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(X_train, n_neighbors=10, include_self=False)
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    agg = AgglomerativeClustering(linkage=linkage,
                                  affinity=affinity,
                                  connectivity=connectivity,
                                  n_clusters=n_clusters)
    agg.fit(X_train)
     
    # Print the clustering scores
    print_scores(y_train, agg.labels_)
    
    labels_palette = sns.color_palette("bright", len(np.unique(agg.labels_)))
    sns.scatterplot(X[:, 0], X[:, 1], hue=agg.labels_, palette=labels_palette)

In [15]:
def clustering_kmeans(X_train, y, old_y, 
                      X_test, y_test, old_y_test,
                      cluster_list,
                      col_names,
                      sel_cls,
                      pca_train=True):
    
    def kmeans_train(X, y, old_y,
                     n_clusters, 
                     n_classes,
                     cluster_palette,
                     row_idx, col_idx,
                     plot=True):

        kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=n_classes)
        kmeans.fit(X)
        
        clust_labels = kmeans.predict(X)        
        cent = kmeans.cluster_centers_
        clust_true_labels = get_cluster_mode(old_y, clust_labels, n_clusters=n_clusters)

        # print the clustering scores 
        print_scores(y, clust_labels, n_clusters=n_clusters)
        
        if plot:
            plot_data(axs, X=X, y=clust_labels,
                      palette=cluster_palette,
                      y_annotate=clust_true_labels,
                      cents=cent,
                      legend=False,
                      row_idx=row_idx, col_idx=col_idx)
        
            return kmeans, col_idx+1
        
        else:
            return kmeans, col_idx
            
    def kmeans_predict(kmeans, X, old_y,
                       n_clusters,
                       cluster_palette,
                       row_idx, col_idx,
                       plot=True):
        
        clust_labels = kmeans.predict(X)
        cent_X = kmeans.cluster_centers_
        clust_true_labels = get_cluster_mode(old_y, clust_labels, n_clusters=n_clusters)

        if plot:
            plot_data(axs, X=X, y=clust_labels,
                      palette=cluster_palette,
                      y_annotate=clust_true_labels,
                      cents=cent_X,
                      legend=False,
                      row_idx=row_idx, col_idx=col_idx)
        
            return clust_labels, col_idx+1
        
        else:
            return clust_labels, col_idx
        
    n_classes = len(sel_cls) 
    plot = True if pca_train else False
    
    if plot:
        fig, axs = get_plot_fig(rows=len(cluster_list),
                                cols=len(col_names),
                                col_names=col_names)
        true_palette = sns.color_palette("bright", n_classes)

    pca = PCA(n_components=2).fit(X_train)
    reduced_train = pca.transform(X_train)
    reduced_test = pca.transform(X_test)
    
    X_train_c, y_train_c, old_y_train_c = get_subset_data(X=X_train,
                                                          y=y,
                                                          old_y=old_y,
                                                          idxs=c_idx)
        
    X_c, y_c, old_y_c = get_subset_data(X=reduced_train,
                                        y=y,
                                        old_y=old_y,
                                        idxs=c_idx)

    X_inc, y_inc, old_y_inc = get_subset_data(X=reduced_train,
                                              y=y,
                                              old_y=old_y,
                                              idxs=inc_idx)
    
    for i, n_clusters in enumerate(cluster_list):
        col_idx = 0
        cluster_palette = sns.color_palette("bright", n_clusters)

        print('***********************n_clusters={}*******************************'.format(n_clusters))
        
        if plot and 'X_train(PCA)' in col_names:
            # Plot the PCA reduced X_train
            cent_train = get_centroids(reduced_train, old_y, n_classes=n_classes)
            plot_data(axs, X=reduced_train, y=old_y,
                      palette=true_palette,
                      y_annotate=sel_cls,
                      cents=cent_train,
                      row_idx=i, col_idx=col_idx)
            col_idx += 1
            
        if 'kmeans trained with X_train' in col_names:
            X = reduced_train if pca_train else X_train
            
            print('Clustering Scores on X_train when trained with X_train')
            kmeans, col_idx = kmeans_train(X, y_train, old_y,
                                           n_clusters=n_clusters,
                                           n_classes=n_classes,
                                           cluster_palette=cluster_palette,
                                           row_idx=i, col_idx=col_idx,
                                           plot=plot)
            print('------------------------------------------------------------------------')
            
        if 'kmeans trained with X_c' in col_names:
            X = X_c if pca_train else X_train_c
            X_train = reduced_train if pca_train else X_train
            
            print('Clustering Scores on X_c when trained with X_c')
            kmeans, col_idx = kmeans_train(X, y_c, old_y_c,
                                           n_clusters=n_clusters,
                                           n_classes=n_classes,
                                           cluster_palette=cluster_palette,
                                           row_idx=i, col_idx=col_idx,
                                           plot=plot)
            print('------------------------------------------------------------------------')
            
            print('Clustering Scores on X_train when trained with X_c')
            # print the clustering scores
            print_scores(y_train, kmeans.predict(X_train), n_clusters=n_clusters)
            print('------------------------------------------------------------------------')
            
        if plot and 'X_inc(PCA)' in col_names:
            # Plot the X_train samples which were misclassified by LSTM
            plot_data(axs, X=X_inc, y=old_y_inc,
                      palette=true_palette,
                      row_idx=i, col_idx=col_idx)
            col_idx += 1

        if 'kmeans predict X_inc' in col_names:
            # Plot cluster predictions on X_inc
            clust_labels, col_idx = kmeans_predict(kmeans, X_inc, old_y_inc,
                                                   n_clusters=n_clusters,
                                                   cluster_palette=cluster_palette,
                                                   row_idx=i, col_idx=col_idx,
                                                   plot=plot)
            
        if plot and 'X_test(PCA)' in col_names:
            # Plot the PCA reduced X_test
            cent_test = get_centroids(reduced_test, y_test, n_classes=n_classes)
            plot_data(axs, X=reduced_test, y=old_y_test,
                      palette=true_palette,
                      y_annotate=sel_cls,
                      cents=cent_test,
                      row_idx=i, col_idx=col_idx)
            col_idx += 1

        if 'kmeans predict X_test' in col_names:
            X = reduced_test if pca_train else X_test
            
            clust_labels_test, col_idx = kmeans_predict(kmeans, X, old_y_test,
                                                        n_clusters=n_clusters,
                                                        cluster_palette=cluster_palette,
                                                        row_idx=i, col_idx=col_idx,
                                                        plot=plot)
                            
            # print the clustering scores 
            print('Clustering Scores on X_test')
            print_scores(y_test, clust_labels_test, n_clusters=n_clusters)
            print('------------------------------------------------------------------------')

In [102]:
method = 'kmeans'
model_dir = '/scratch/sk7898/pedbike/models/lstm/'
cls_str_list = ['1_2_3_4']
sel_cls_list = [[1, 2, 3, 4]]
layer_name = 'counting_dense_2'
pca_train = False

for idx, (cls_str, sel_cls) in enumerate(zip(cls_str_list, sel_cls_list)):
    model_str = os.path.join(cls_str + '_amp_512_hidden_128/best_model.h5')
    model_path = os.path.join(model_dir, model_str)

    X_train, X_test, y_train, y_test, old_y, old_y_test, _, _ = get_fft_data(sel_cls=sel_cls,
                                                                             data_mode='amp')
    
    X = X_train
    y = y_train
    old_y = old_y.flatten()
    old_y_test = old_y_test.flatten()
    y_test = y_test.flatten()
    model = load_model(model_path)
    pred = model.predict(x=X)
    cls_pred = np.argmax(pred, axis = 1)
    c_idx, inc_idx, c_lst, inc_lst = get_correct_incorrect_idx(y, 
                                                               cls_pred, 
                                                               n_classes=len(sel_cls)) 
    
    model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

    emb_train = model.predict(x=X)
    emb_test = model.predict(x=X_test)
    
    if method == 'kmeans':
        cluster_list = [20, 50]
        col_names = [#'X_train(PCA)',
                     #'kmeans trained with X_train',
                     'kmeans trained with X_c',
                     #'kmeans predict X_test'
                    ]

        clustering_kmeans(emb_train, y, old_y,
                          emb_test, y_test, old_y_test,
                          cluster_list=cluster_list,
                          col_names=col_names,
                          sel_cls=sel_cls,
                          pca_train=pca_train
                         )
        
    if method == 'DBSCAN':
        min_samples_lst = [2, 5, 10]
        eps_lst = [0.3, 0.5, 1.0, 1.5]
        clustering_dbscan(emb_train, y,
                          min_samples_lst=min_samples_lst,
                          eps_lst=eps_lst,
                          pca_train=pca_train
                         )
        
    if method == 'Agglomerative':
        clustering_agglomerative(emb_train, y,
                                 affinity='euclidean',
                                 linkage='ward',  
                                 n_clusters=8,
                                 pca_train=pca_train
                                )

***********************n_clusters=20*******************************
Clustering Scores on X_c when trained with X_c
Homogeneity: 0.9493304975436917
Homogeneous Clusters:  7
Mutual Information:  1.2967430690207222
------------------------------------------------------------------------
Clustering Scores on X_train when trained with X_c
Homogeneity: 0.8739611243814985
Homogeneous Clusters:  1
Mutual Information:  1.1927021627826266
------------------------------------------------------------------------
***********************n_clusters=50*******************************
Clustering Scores on X_c when trained with X_c
Homogeneity: 0.9813594610560762
Homogeneous Clusters:  36
Mutual Information:  1.3404932029836214
------------------------------------------------------------------------
Clustering Scores on X_train when trained with X_c
Homogeneity: 0.9030247232992605
Homogeneous Clusters:  7
Mutual Information:  1.2323655028562408
------------------------------------------------------------